In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel("GSAF5.xls")

EDA was already done, are fair understanding of what need to be cleaned exists already.

In [3]:
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_").str.replace("/","_",regex=False).str.strip("_")
df.columns

Index(['date', 'year', 'type', 'country', 'state', 'location', 'activity',
       'name', 'sex', 'age', 'injury', 'fatal_y_n', 'time', 'species',
       'source', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1',
       'original_order', 'unnamed:_21', 'unnamed:_22'],
      dtype='object')

In [4]:
df['country'] = (df['country'].str.strip().str.upper())

In [5]:
df['country']= df['country'].replace({'CEYLON (SRI LANKA)':'SRI LANKA','CEYLON': 'SRI LANKA','UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES', 'AFRICA': 'COAST OF AFRICA', 'BURMA': 'MYANMAR',
    'WESTERN SAMOA': 'SAMOA', 'TURKS & CAICOS': 'TURKS AND CAICOS',
    'TRINIDAD & TOBAGO': 'TRINIDAD AND TOBAGO',
    'ST. MARTIN': 'ST MARTIN',
    'ST. MAARTIN': 'ST MARTIN'})

In [6]:
water_keywords = [
    'OCEAN', 'SEA', 'GULF', 'BAY', 'COAST', 'ATLANTIC', 'PACIFIC',
    'INDIAN', 'MEDITERRANEAN', 'CARIBBEAN'
]

mask_water = df['country'].str.contains(
    '|'.join(water_keywords),
    na=False
)

df.loc[mask_water, 'country']

#rows can later become:

# country = NaN
# region = "PACIFIC OCEAN"

,country
149,CORAL SEA
344,BRITISH OVERSEAS TERRITORY
833,"ST HELENA, BRITISH OVERSEAS TERRITORY"
874,"ST HELENA, BRITISH OVERSEAS TERRITORY"
1174,ATLANTIC OCEAN
...,...
6897,MEDITERRANEAN SEA
6984,RED SEA?
6990,MEDITERRANEAN SEA
7013,NORTH PACIFIC OCEAN


In [7]:
df.loc[mask_water, 'country'] = pd.NA


In [8]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

/tmp/ipython-input-2370506791.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [9]:
df['fatal_y_n'] = df['fatal_y_n'].str.lower().str.strip()

In [10]:
df['fatal_y_n'] = df['fatal_y_n'].replace({'nan': 'unknown', 'm':'n', 'f': 'unkown','nq':'unknown','y x 2':'y' })

In [11]:
df['sex'] = df['sex'].str.lower().str.strip()

In [12]:
df['sex'] = df['sex'].replace({'m x 2':'m','lli':'unknown', '.':'unknown', 'n':'m'})

Deleting all columns after species, as they dont offer and insigts

In [13]:
df = df.loc[:, :'species']


In [14]:
df['species'].unique()

array(['3m Bull shark ', 'Great White Shark', '3m shark', ...,
       "12' tiger shark", 'Blue pointers',
       'Said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'],
      dtype=object)

In [15]:
df['species'] = df['species'].str.lower().str.strip().str.replace('"', '', regex=False)

In [16]:
df['species'] = df['species'].str.replace(
    r"\b\d+[,\d]*\s*(lb|lbs|kg|m|ft|feet|')\b",
    "",
    regex=True
)

In [17]:
df['species'] = df['species'].str.extract(r'([a-z\s-]*shark)',expand=False)

In [18]:
df['species'].unique()

array([' bull shark', 'great white shark', ' shark', nan,
       'suspected tiger shark', 'nurse shark', 'tiger or bull shark',
       'tiger shark', 'lemon shark', 'undetermined small shall shark',
       'sand tiger shark', 'small shark', 'bull shark', 'dusky shark',
       ' tiger shark', 'mako shark', 'reportedly tiger or bull shark',
       'reportedly a tiger shark', 'reported as a large tiger shark',
       'large tiger shark', 'reportedly tiger shark', 'reef shark',
       ' white shark', '- blacktip shark',
       'not stated area has tiger and galapogas shark',
       'not specified bull shark', 'large shark', ' sandbar shark',
       'carribean reef shark', 'blacktip reef shark', 'white shark',
       'raggedtooth shark', 'sevengill shark', 'shark',
       'oceanic whitetip shark', ' galapagos shark', 'cookiecutter shark',
       'blue shark', 'wfite shark', 'wobbegong shark',
       'caribbean rreef shark', 'grey reef shark', 'caribbean reef shark',
       ' horn shark', 'b

In [19]:
df['species'] = (
    df['species']
      .astype(str)
      .str.lower()
      .str.replace('\xa0', ' ', regex=False)
      .str.replace('"', '', regex=False)
      .str.strip())

In [20]:
df['species'] = df['species'].str.replace(
    r"\b\d+[,\d]*\s*(kg|lb|lbs|cm|m|ft|feet|')\b",
    "",
    regex=True)

not_shark_patterns = r'not a shark|no shark|injuries not caused by a shark|local authorities speculated'

df.loc[df['species'].str.contains(not_shark_patterns, na=False),
    'species'] = 'NOT A SHARK'

In [21]:
df['species'] = df['species'].str.extract(
    r'([a-z\s\-]+shark)',
    expand=False)

In [22]:
df['species'] = df['species'].replace(
    {
    'bull shark': 'BULL SHARK',
    'zambezi shark': 'BULL SHARK',

    'great white shark': 'GREAT WHITE SHARK',
    'white shark': 'GREAT WHITE SHARK',

    'tiger shark': 'TIGER SHARK',

    'blacktip shark': 'BLACKTIP SHARK',
    'black tipped shark': 'BLACKTIP SHARK',

    'grey nurse shark': 'GREY NURSE SHARK',
    'gray nurse shark': 'GREY NURSE SHARK',
    'raggedtooth shark': 'GREY NURSE SHARK',

    'lemon shark': 'LEMON SHARK',
    'spinner shark': 'SPINNER SHARK',
    'hammerhead shark': 'HAMMERHEAD SHARK',
    'reef shark': 'REEF SHARK',
    'dusky shark': 'DUSKY SHARK',
    'sand tiger shark': 'SAND TIGER SHARK',
    'sandtiger shark': 'SAND TIGER SHARK',
    'mako shark': 'MAKO SHARK',
    'blue shark': 'BLUE SHARK',
    'whitetip shark': 'OCEANIC WHITETIP SHARK',
    'oceanic whitetip shark': 'OCEANIC WHITETIP SHARK',
    'sevengill shark': 'SEVENGILL SHARK',
    'seven gill shark': 'SEVENGILL SHARK',
    'shark': 'UNKNOWN SHARK'
}
)

In [23]:
df['species'].unique()

array(['BULL SHARK', 'GREAT WHITE SHARK', nan, 'suspected tiger shark',
       'nurse shark', 'tiger or bull shark', 'TIGER SHARK', 'LEMON SHARK',
       'undetermined small shall shark', 'SAND TIGER SHARK',
       'small shark', 'DUSKY SHARK', 'MAKO SHARK',
       'reportedly tiger or bull shark', 'reportedly a tiger shark',
       'reported as a large tiger shark', 'large tiger shark',
       'reportedly tiger shark', 'REEF SHARK', '- blacktip shark',
       'not stated area has tiger and galapogas shark',
       'not specified bull shark', 'large shark', 'sandbar shark',
       'carribean reef shark', 'blacktip reef shark', 'GREY NURSE SHARK',
       'SEVENGILL SHARK', 'OCEANIC WHITETIP SHARK', 'galapagos shark',
       'cookiecutter shark', 'BLUE SHARK', 'wfite shark',
       'wobbegong shark', 'caribbean rreef shark', 'grey reef shark',
       'caribbean reef shark', 'horn shark', 'bronze whaler shark',
       'to  shark', 'a small shark', 'tiger  shark',
       'juvenile white sh

In [24]:
unclear_patterns = (
    r'suspected|possibly|thought to involve|reported to involve|'
    r'reported as|said to involve|allegedly|believed|'
    r'\bor\b|unknown|unidentified|small shark|large shark|'
    r'several shark|pack of shark'
)

df.loc[
    df['species'].str.contains(unclear_patterns, na=False),
    'species'
] = 'UNKNOWN SHARK'

In [25]:
df['species'] = df['species'].replace(
    {
    'bull shark': 'BULL SHARK',
    'zambesi shark': 'BULL SHARK',

    'white shark': 'GREAT WHITE SHARK',
    'great white shark': 'GREAT WHITE SHARK',

    'tiger shark': 'TIGER SHARK',
    'blacktip shark': 'BLACKTIP SHARK',
    'spinner shark': 'SPINNER SHARK',
    'hammerhead shark': 'HAMMERHEAD SHARK',
    'lemon shark': 'LEMON SHARK',
    'reef shark': 'REEF SHARK',
    'grey nurse shark': 'GREY NURSE SHARK',
    'gray nurse shark': 'GREY NURSE SHARK',
    'mako shark': 'MAKO SHARK',
    'blue shark': 'BLUE SHARK',
    'sand tiger shark': 'SAND TIGER SHARK',
    'dusky shark': 'DUSKY SHARK',
    'oceanic whitetip shark': 'OCEANIC WHITETIP SHARK',
    'sevengill shark': 'SEVENGILL SHARK',
    'shark': 'UNKNOWN SHARK'
}
)

In [26]:
df['species'].value_counts()

,count
species,
GREAT WHITE SHARK,661
TIGER SHARK,282
BULL SHARK,227
UNKNOWN SHARK,209
GREY NURSE SHARK,81
...,...
ground shark,1
remains recovered from shark,1
description of shark,1


In [27]:
allowed_species = {
    'bull shark': 'BULL SHARK',
    'great white shark': 'GREAT WHITE SHARK',
    'white shark': 'GREAT WHITE SHARK',
    'tiger shark': 'TIGER SHARK',
    'blacktip shark': 'BLACKTIP SHARK',
    'spinner shark': 'SPINNER SHARK',
    'reef shark': 'REEF SHARK',
    'grey nurse shark': 'GREY NURSE SHARK',
    'gray nurse shark': 'GREY NURSE SHARK',
    'hammerhead shark': 'HAMMERHEAD SHARK',
    'lemon shark': 'LEMON SHARK',
    'mako shark': 'MAKO SHARK',
    'dusky shark': 'DUSKY SHARK',
    'sand tiger shark': 'SAND TIGER SHARK',
    'blue shark': 'BLUE SHARK',
    'oceanic whitetip shark': 'OCEANIC WHITETIP SHARK',
    'sevengill shark': 'SEVENGILL SHARK'
}

In [28]:
def strict_species_classifier(s):
    if pd.isna(s):
        return pd.NA

    s = str(s).lower().strip()

    # anything with uncertainty / junk → UNKNOWN
    unclear_patterns = (
        r'suspected|possibly|reported|thought|said|alleged|believed|'
        r'juvenile|small|large|young|female|male|'
        r'\bor\b|unknown|unidentified|'
        r'\d|kg|lb|cm|m|ft|-'
    )

    if re.search(unclear_patterns, s):
        return 'UNKNOWN SHARK'

    # exact species match only
    for k, v in allowed_species.items():
        if s == k:
            return v

    # everything else
    if 'shark' in s:
        return 'UNKNOWN SHARK'

    return pd.NA

In [29]:
import re

df['species'] = df['species'].apply(strict_species_classifier)

In [30]:
df['species'].unique()

array(['BULL SHARK', 'GREAT WHITE SHARK', <NA>, 'UNKNOWN SHARK',
       'TIGER SHARK', 'SAND TIGER SHARK', 'DUSKY SHARK', 'REEF SHARK',
       'GREY NURSE SHARK', 'SEVENGILL SHARK', 'OCEANIC WHITETIP SHARK',
       'BLUE SHARK', 'BLACKTIP SHARK', 'SPINNER SHARK'], dtype=object)

In [31]:
df.head(1)

,date,year,type,country,state,location,activity,name,sex,age,injury,fatal_y_n,time,species
0,NaT,2025.0,Unprovoked,AUSTRALIA,NSW,Crowdy Bay,Swimming,Lukas Schindler,m,26,Serious leg injuries,n,0630hrs,BULL SHARK


In [32]:
df['type'].unique()

array(['Unprovoked', 'Provoked', 'Questionable', 'unprovoked',
       ' Provoked', 'Watercraft', 'Sea Disaster', nan, '?', 'Unconfirmed',
       'Unverified', 'Invalid', 'Under investigation', 'Boat'],
      dtype=object)

In [33]:
df['type'] = (
    df['type']
      .astype(str)
      .str.lower()
      .str.strip()
)

In [34]:
def clean_type(t):
    if t == 'nan' or pd.isna(t):
        return 'UNKNOWN'

    if 'unprovoked' in t:
        return 'UNPROVOKED'

    if 'provoked' in t:
        return 'PROVOKED'

    if 'boat' in t:
        return 'BOAT'

    if 'sea disaster' in t or 'ship' in t:
        return 'SEA DISASTER'

    if 'invalid' in t:
        return 'INVALID'

    return 'UNKNOWN'

In [35]:
df['type'] = df['type'].apply(clean_type)

In [36]:
df['type'].unique()

array(['UNPROVOKED', 'PROVOKED', 'UNKNOWN', 'SEA DISASTER', 'INVALID',
       'BOAT'], dtype=object)

In [37]:
df = df.drop_duplicates()


In [38]:
df.isna().mean().sort_values(ascending=False)*100

,0
species,66.520198
time,49.936215
age,42.395464
date,12.870305
activity,8.291991
sex,8.206945
location,8.022679
fatal_y_n,7.965982
state,6.888731
name,3.104181


Time will be droped since almost 50% missing and not relevant for analysis

In [39]:
df = df.drop(columns=['time'])

In [40]:
df['activity'] = df['activity'].str.strip().fillna('UNKNOWN')

In [42]:
df['fatal_y_n'] = df['fatal_y_n'].str.upper().str.strip()
df['fatal_y_n'] = df['fatal_y_n'].replace({'Y': 'FATAL', 'N': 'NON-FATAL', '': 'UNKNOWN'})

In [43]:
df['fatal_y_n'] = df['fatal_y_n'].str.strip().fillna('UNKNOWN')

In [46]:
df = df.rename(columns={'fatal_y_n':'fatality'})

In [47]:
df.head()

,date,year,type,country,state,location,activity,name,sex,age,injury,fatality,species
0,NaT,2025.0,UNPROVOKED,AUSTRALIA,NSW,Crowdy Bay,Swimming,Lukas Schindler,m,26,Serious leg injuries,NON-FATAL,BULL SHARK
1,NaT,2025.0,UNPROVOKED,AUSTRALIA,NSW,Crowdy Bay,Swimming,Livia Mulheim,f,25,Not stated,FATAL,BULL SHARK
2,NaT,2025.0,UNPROVOKED,AUSTRALIA,Western Australia,Prevelly Beach Magaret River,Foil Boarding,Andy McDonald,m,61,No Injury to self,NON-FATAL,GREAT WHITE SHARK
3,NaT,2025.0,UNPROVOKED,FRENCH POLYNESIA,Marquesas Islands,Hakahau Bay,Swimming,Not stated (Dentist),m,40,Deep Gash to bicep,NON-FATAL,<NA>
4,NaT,2025.0,UNPROVOKED,USA,Hawaii,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,m,?,Injuries to legs,NON-FATAL,<NA>


In [48]:
df.shape

(7055, 13)

In [49]:
df.to_excel("shark_attack_cleaned.xlsx", index=False)